In [ ]:
import requests
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import gamma
from sklearn import linear_model
import numpy as np
from datetime import datetime
import math

In [ ]:
import c3aidatalake

In [ ]:
# Returns datetime object of the given date string
# Compare dates with >, <, or ==
def timeFormat(s):
    if str(s) == "nan":
        return s
    else:
        return datetime.strptime(s, "%Y-%m-%dT%H:%M:%SZ")

# Returns float timestamp (seconds since 1 Jan 1970)
# Compare timestamps with >, <, or ==
def timestampFormat(s):
    if str(s) == "nan":
        return s
    else:
        return timeFormat(s).timestamp()

# How to convert column of date strings to these above formats:
# DFtimeFormat(dataframe name, string of column header which contains the time strings)
def DFtimeFormat(df, column_of_timestrings):
    df[column_of_timestrings] = df[column_of_timestrings].apply(timeFormat)
    return

In [ ]:
# fetching state policies
statePolicies = c3aidatalake.fetch(
    "locationpolicysummary", 
    {
        "spec" : {
            "limit" : -1
        }
    }).drop(44)

Generating numerical columns from statePolicies:

In [ ]:
# dicts used to quantify governemnt policy by restrictiveness 
# (0 - not restrictive -> higher - more restrictive)
quantifyDicts = {}
quantifyDicts["easingOrder"] = {
    "Reopened" : 0, 
    "Proceeding with Reopening" : 1,
    "Paused" : 2, 
    "New Restrictions Imposed" : 3
}
quantifyDicts["stayAtHome"] = {
    "No Action" : 0,
    "Lifted" : 0,
    "Rolled Back to High Risk Groups" : 1,
    "New Stay at Home Order" : 2,
    "Statewide" : 2
}
quantifyDicts["mandatoryQuarantine"]  = {
    "No Action" : 0,
    "Lifted" : 0,
    "From Certain States (New)" : 1,
    "Rolled Back to Certain States" : 1,
    "From Certain States" : 1,
    "Rolled Back to International Travel" : 2,
    "All Travelers" : 3
}
quantifyDicts["nonEssentialBusiness"] = {
    "No Action" : 0,
    "All Non-Essential Businesses Permitted to Reopen" : 0,
    "Some Non-Essential Businesses Permitted to Reopen" : 1,
    "All Non-Essential Businesses Permitted to Reopen with Reduced Capacity" : 1,
    "Some Non-Essential Businesses Permitted to Reopen with Reduced Capacity" : 2,
    "New Business Closures or Limits" : 3
}
quantifyDicts["largeGatherings"] = {
    "Lifted" : 0,
    "No Action" : 0,
    "Expanded to New Limit Above 25" : 1,
    "New Limit on Large Gatherings in Place" : 1,
    "Expanded to New Limit of 25 or Fewer" : 2,
    ">10 People Prohibited" : 3,
    "All Gatherings Prohibited" : 4
}
quantifyDicts["schoolClosure"] = {
    "Rescinded" : 0,
    "Recommended Closure for School Year" : 1,
    "Recommended Closure" : 2,
    "Closed for School Year" : 3,
    "Closed" : 4
}
quantifyDicts["restaurantLimit"] = {
    "No Action" : 0,
    "Reopened to Dine-in Service" : 1,
    "Reopened to Dine-in Service with Capacity Limits" : 2,
    "New Service Limits" : 3,
    "Newly Closed to Dine-in Service" : 3
}
quantifyDicts["barClosures"] = {
    "Reopened" : 0,
    "New Service Limits" : 1,
    "Closed" : 2,
    "Newly Closed" : 2
}
quantifyDicts["faceCoveringRequirement"] = {
    "No" : 0,
    "Required for Certain Employees" : 1,
    "Allows Local Officals to Require for General Public" : 1,
    "Required for Certain Employees; Allows Local Officials to Require for General Public" : 1,
    "Required for General Public" : 2
}
quantifyDicts["PrimaryElectionPostponement"] = {
    "No" : 0,
    "Postponed" : 1,
    "Canceled" : 2
}
quantifyDicts["emergencyDeclaration"] = {
    "Yes" : 0
}
quantifyDicts["waiveTreatmentCost"]  = {
    "No Action" : 0,
    "State-Insurer Agreement" : 1,
    "State Requires" : 2
}  
quantifyDicts["freeVaccine"] = {
    "No Action" : 0,
    "State-Insurer Agreement" : 1,
    "State Requires" : 2
}
quantifyDicts["waiverOfPriorAuthorizationRequirements"] = {
    "No Action" : 0,
    "For COVID-19 Testing" : 1,
    "For COVID-19 Testing and Treatment" : 2
}
quantifyDicts["prescriptionRefill"] = {
    "No Action" : 0,
    "Expired" : 1,
    "State Requires" : 2
}
quantifyDicts["premiumPaymentGracePeriod"] = {
    "No Action" : 0,
    "Expired" : 1,
    "COVID-19 Diagnosis/Impacts Only" : 2,
    "Grace Period Extended for All Individual Policies" : 3,
    "All Policies" : 4
}
quantifyDicts["marketplaceSpecialEnrollmentPeriod"] = {
    "No" : 0,
    "Ended" : 1,
    "Active" : 2
}
quantifyDicts["section1135Waiver"] = {
    "Unapproved" : 0,
    "Approved" : 1
}
quantifyDicts["paidSickLeaves"] = {
    "No Action" : 0,
    "Proposed - March 2020" : 1,
    "Enacted" : 2
}
quantifyDicts["expandsAccesstoTelehealthServices"] = {
    "No" : 0,
    "Yes" : 1
}

In [ ]:
# generates mapper for each column using dicts defined above
def mapperGenerator(colName):
    if colName in quantifyDicts.keys():
        def mapper(val):
            return quantifyDicts[colName][val]
    else:
        def mapper(val):
            return val
    return mapper

In [ ]:
# read in stategdp info
# adding quarter 2 gdp change as a column to statepolicies
import re
stategdp = pd.read_excel(io = "qgdpstate1020_0.xlsx", index_col = 0, header = 1)
Q2GDPChange = []
for id in statePolicies["id"]:
    m = re.match(r"(.+)_UnitedStates_Policy$", id)
    Q2GDPChange.append(stategdp["2020Q2"][m[1]])
statePolicies.insert(len(statePolicies.columns), "Q2GDPChange", Q2GDPChange)

In [ ]:
# converting state policies time stamps
DFtimeFormat(statePolicies, "lastSavedTimestamp")

In [ ]:
# Q2END is comparison datetime object to find policies implemented before end of 2nd quarter
Q2END = datetime(2020, 7, 1)
# columns to be used in relevantPolicies
newColumns = ['id', 'easingOrder', 'stayAtHome', 'mandatoryQuarantine', 'nonEssentialBusiness', 'largeGatherings', 
              'schoolClosure', 'restaurantLimit', 'barClosures', 'faceCoveringRequirement']

In [ ]:
# columns are newColumns + versionDate + Q2GDPChange
# create new data frame
# for each state in statePolicies, try to access relevant dates, or use info from statePolicies
# append to data frame
relevantPolicies = []
for i in statePolicies.index:
    data = []
    try:
        allStateVersions = c3aidatalake.read_data_json(
            "locationpolicysummary",
            "allversionsforpolicy",
            body = {
                "this" : {
                    "id" : statePolicies["id"][i]
                }
            }
        )
        allStateVersions = pd.json_normalize(allStateVersions)
        DFtimeFormat(allStateVersions, "versionDate")
        relevantPolicyFound = False
        for ind in allStateVersions.index:
            if allStateVersions["versionDate"][ind] < Q2END:
                data = [allStateVersions[col][ind] for col in newColumns]
                data.append(allStateVersions["versionDate"][ind])
                relevantPolicyFound = True
                break
        if not relevantPolicyFound:
            raise 
    except:
        data = [statePolicies[col][i] for col in newColumns]
        data.append(statePolicies["lastSavedTimestamp"][i])
    data.append(statePolicies["Q2GDPChange"][i])
    relevantPolicies.append(data)
relevantPolicies = pd.DataFrame(relevantPolicies, columns = newColumns + ["versionDate", "Q2GDPChange"])

In [ ]:
# independent variables for multiple linear regression
xVars = ['easingOrder', 'stayAtHome', 'mandatoryQuarantine', 'nonEssentialBusiness', 'largeGatherings', 
         'schoolClosure', 'restaurantLimit', 'barClosures', 'faceCoveringRequirement']

In [ ]:
# quantify relevantPolicies
for col in relevantPolicies.columns:
    mper = mapperGenerator(col)
    relevantPolicies[col] = relevantPolicies[col].apply(mper)

In [ ]:
relevantX = relevantPolicies[xVars]
relevantY = relevantPolicies["Q2GDPChange"]
relevantRegr = linear_model.LinearRegression()
relevantRegr.fit(relevantX, relevantY)
for i in range(len(xVars)):
    print("Coefficient of ", xVars[i], ": ", relevantRegr.coef_[i])
print("Intercept: ", relevantRegr.intercept_)
print("R^2: ", relevantRegr.score(relevantX, relevantY))


plt.xticks(rotation='vertical')
pyplot.bar([x for x in range(len(relevantRegr.coef_))], relevantRegr.coef_, tick_label=xVars)
pyplot.show()

In [ ]:
# Correlations of relevant policies with GDP change
plt.title("Policies to GDP Correlations")
plt.xticks(rotation='vertical')
labels=['easingOrder', 'stayAtHome', 'mandatoryQuarantine',
       'nonEssentialBusiness', 'largeGatherings', 'schoolClosure',
       'restaurantLimit', 'barClosures', 'faceCoveringRequirement', 'restrictivenessMeasure']
pyplot.bar([x for x in range(len(labels))], relevantPolicies.corr()["Q2GDPChange"].drop("Q2GDPChange"), tick_label=labels)
pyplot.show()

In [ ]:
# Number of trials to fit model and average over
FITS = 100

# decision tree for feature importance on a regression problem
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from matplotlib import pyplot
# define dataset
X, y = relevantX, relevantY

# define the model
model = DecisionTreeRegressor(max_depth=4)
# fit the model
avg_importance = [0 for x in range(len(xVars))]
for i in range(FITS):
	model.fit(X, y)
	avg_importance += model.feature_importances_

avg_importance /= FITS

# summarize feature importance
for i,v in enumerate(avg_importance):
	print(xVars[i], v)

print("Intercept: ", model.predict([[0 for x in range(len(xVars))]])[0])
print("R^2: ", model.score(X,y))

In [ ]:
# Number of trials to fit model and average over
FITS = 100
# random forest for feature importance on a regression problem
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot
# define dataset
X, y = relevantX, relevantY
# define the model
model = RandomForestRegressor(max_depth=6)
# fit the model
avg_importance2 = [0 for x in range(len(xVars))]
for i in range(FITS):
	model.fit(X, y)
	avg_importance2 += model.feature_importances_

avg_importance2 /= FITS

# summarize feature importance
for i,v in enumerate(avg_importance2):
	print(xVars[i], v)

print("Intercept: ", model.predict([[0 for x in range(len(xVars))]])[0])
print("R^2: ", model.score(X,y))

In [ ]:
# Plotting random forest and decision tree feature importances side by side
plt.xticks(rotation='vertical')
plt.title("Decision Tree")
pyplot.bar(range(len(xVars)), avg_importance, tick_label=xVars)
plt.show()
plt.xticks(rotation='vertical')
plt.title("Random Forest")
pyplot.bar(range(len(xVars)), avg_importance2, tick_label=xVars)
plt.show()

In [ ]:
results = permutation_importance(model, X, y, scoring='neg_mean_squared_error')
# get importance
importance = results.importances_mean
# summarize feature importance
for i,v in enumerate(importance):
	print(xVars[i], v)
# plot feature importance
plt.xticks(rotation='vertical')
pyplot.bar(range(len(xVars)), importance, tick_label=xVars)
pyplot.show()